In [1]:
import sys, os
import h5py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../src/')
from tensor import Particles

In [2]:
hf = h5py.File('../data/elucid_noQ_mock.hdf5', 'r')['particles']
idx = hf['cen_flag'] == 0
pos = np.stack((hf['x'][idx], hf['y'][idx], hf['z'][idx]), axis=1)
mass = hf['host_lgmass'][idx]
del hf

In [3]:
# parameters
ngrid = 256                  # number of grids
Lbox = 500                   # box size, ckpc/h
Rs = 1                       # smoothing scale (FWHM), ckpc/h
workers = os.cpu_count() - 2 # number of CPU for FFT parallel computing
print(f'No. CPU: {workers}')

No. CPU: 6


In [4]:
# initialize
grid = Particles(ngrid, Lbox)

# assign particle to mesh
grid.assign(pos, mass)

# tensor field calculation
grid.TensorField(Rs, workers=workers)

In [5]:
# interpolate tensor at given positions
x = np.array([10, 20])
y = np.array([300, 300])
z = np.array([60, 60])
xyz = np.stack((x, y, z), axis=1)
tensor = grid.TensorInterp(xyz)

In [6]:
# use update to calculate by a single command
%time grid.TensorField(Rs, workers=workers, update=True, pos=pos, mass=mass)

CPU times: user 11.3 s, sys: 19.6 s, total: 30.9 s
Wall time: 31.9 s
